In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import precision_recall_curve, roc_curve, auc

In [2]:
# Command to use the full widht of the screen
#from IPython.core.display import display, HTML
#display(HTML("<style>.container { width:100% !important; }/style>"))

In [3]:
def lcls(x, links):
    s = links.shape[0]
    lcl = np.zeros(s)
    nodes_r = np.unique(links[:,0])
    nodes_c = np.unique(links[:,1])
    ne_r = dict()
    ne_c = dict()
    for i in range(x.shape[0]):
        if any(i==nodes_r):
            ne_c[i] = np.argwhere(x[i,:])
    for i in range(x.shape[1]):
        if any(i==nodes_c):
            ne_r[i] = np.argwhere(x[:,i])
    for i in range(s):
        n_r = ne_r[links[i,1]].flatten()
        n_c = ne_c[links[i,0]].flatten()
        if x[links[i,0], links[i,1]]:
            n_r = n_r[n_r != links[i,0]]
            n_c = n_c[n_c != links[i,1]]
        sx = x[n_r][:,n_c]
        lcl[i] = np.sum(sx)
    return lcl

In [4]:
def create_LCL(S, links, l_param):
    if links[0].size != 0:
        lcl = lcls(S, np.array(links).T)
        p_lcl = lcl/np.sum(lcl)
        if np.isnan(p_lcl).all():
            if l_param < links[0].size:
                rand_edge = np.random.choice(links[0].size, size=l_param, replace=False)
            else:
                rand_edge = range(links[0].size)
        else:
            if l_param > np.count_nonzero(p_lcl):
                rand_edge = np.random.choice(links[0].size, size=np.count_nonzero(p_lcl), p=p_lcl, replace=False)
            else:
                rand_edge = np.random.choice(links[0].size, size=l_param, p=p_lcl, replace=False)
        a = links[0][rand_edge]
        b = links[1][rand_edge]
        S[a,b] =  1

In [5]:
def BALCL(init_nodes_x, final_nodes_x, init_nodes_y, final_nodes_y, m_param_x, m_param_y, l_param, Initial_Connectivity='all-to-all', LCL=True):
    assert init_nodes_x > 1
    assert init_nodes_y > 1
    assert init_nodes_x < final_nodes_x
    assert init_nodes_y < final_nodes_y
    assert init_nodes_x >= m_param_x
    assert init_nodes_y >= m_param_y

    #INITIAL NETWORK:
    A = np.zeros((final_nodes_x, final_nodes_y))
    if Initial_Connectivity == 'all-to-all':
        for i in range(init_nodes_x):
            for j in range(init_nodes_y):
                A[i,j] = 1
    elif Initial_Connectivity == 'my_random':
        for i in range(init_nodes_x):
            for j in range(init_nodes_y):
                A[i,j] = int(np.random.random()+0.5)
            if not A[i].any() != 0:
                #print("Empty row:", i, A[i])
                rand_column = np.random.choice(range(init_nodes_y))
                A[i, rand_column] = 1
        for j in range(init_nodes_y):
            if not A[:,j].any() != 0:
                #print("Empty column:", j, A[:,j])
                rand_row = np.random.choice(range(init_nodes_x))
                A[rand_row, j] = 1
    #print("Initial A")
    #print(A)
    
    #NETWORK EVOLUTION:
    X = np.arange(init_nodes_x, final_nodes_x)
    Y = np.arange(init_nodes_y, final_nodes_y)
    x = len(X)
    y = len(Y)
    if x >= y:
        c = x/y
        for i in range(y):
            for j in range(int(c)):
                idx = j+i*int(c)
                p_deg_target = np.sum(A[:,:Y[i]], axis=0)
                p_deg_target /= np.sum(p_deg_target)
                rand_target = np.random.choice(Y[i], size=m_param_x, p=p_deg_target, replace=False)
                A[X[idx], rand_target] = 1
                if LCL:
                    S = A[:X[idx]+1, :Y[i]]
                    missing = np.nonzero(S==0)
                    create_LCL(S, missing, l_param)
            p_deg_source = np.sum(A[:X[idx]+1,:], axis=1)
            p_deg_source /= np.sum(p_deg_source)
            rand_source = np.random.choice(X[idx]+1, size=m_param_y, p=p_deg_source, replace=False)
            A[rand_source, Y[i]] = 1
            if LCL:
                S = A[:X[idx]+1, :Y[i]+1]
                missing = np.nonzero(S==0)
                create_LCL(S, missing, l_param)
        for k in range(idx+1, len(X)):
            p_deg_target = np.sum(A[:,:Y[i]], axis=0)
            p_deg_target /= np.sum(p_deg_target)
            rand_target = np.random.choice(Y[i], size=m_param_x, p=p_deg_target, replace=False)
            A[X[k], rand_target] = 1
            if LCL:
                S = A[:X[k]+1, :Y[i]+1]
                missing = np.nonzero(S==0)
                create_LCL(S, missing, l_param)
    else:
        c = y/x
        for i in range(x):
            for j in range(int(c)):
                idx = j+i*int(c)
                p_deg_source = np.sum(A[:X[i],:], axis=1)
                p_deg_source /= np.sum(p_deg_source)
                rand_source = np.random.choice(X[i], size=m_param_y, p=p_deg_source, replace=False)
                A[rand_source, Y[idx]] = 1
                if LCL:
                    S = A[:X[i], :Y[idx]+1]
                    missing = np.nonzero(S==0)
                    create_LCL(S, missing, l_param)
            p_deg_target = np.sum(A[:,:Y[idx]+1], axis=0)
            p_deg_target /= np.sum(p_deg_target)
            rand_target = np.random.choice(Y[idx]+1, size=m_param_x, p=p_deg_target, replace=False)
            A[X[i], rand_target] = 1
            if LCL:
                S = A[:X[i]+1, :Y[idx]+1]
                missing = np.nonzero(S==0)
                create_LCL(S, missing, l_param)
        for k in range(idx+1, len(Y)):
            p_deg_source = np.sum(A[:X[i],:], axis=1)
            p_deg_source /= np.sum(p_deg_source)
            rand_source = np.random.choice(X[i], size=m_param_y, p=p_deg_source, replace=False)
            A[rand_source, Y[k]] = 1
            if LCL:
                S = A[:X[i]+1, :Y[k]+1]
                missing = np.nonzero(S==0)
                create_LCL(S, missing, l_param)

    return A

In [6]:
testBALCL = BALCL(3, 20, 2, 16, 1, 1, 2, Initial_Connectivity='all-to-all', LCL=True)
testBALCL

array([[1., 1., 1., 1., 1., 1., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0.],
       [1., 1., 1., 1., 1., 1., 0., 1., 0., 1., 0., 0., 0., 0., 0., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 1., 0., 1., 0.],
       [1., 1., 1., 1., 1., 1., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0.],
       [1., 1., 1., 1., 1., 1., 0., 1., 0., 1., 0., 0., 0., 0., 0., 1.],
       [1., 1., 1., 1., 1., 0., 0., 1., 1., 0., 0., 1., 0., 0., 0., 0.],
       [1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [1., 1., 1., 1., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 1., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,